# Taxi Fare Prediction Modeling

## Introduction

### Research Objective
The primary objective of this study is to develop a predictive fare model using the comprehensive NYC yellow taxi trip dataset from 2023. The aim is to identify key fare determinants and leverage advanced machine learning techniques to create a model that can estimate taxi fares accurately. This model will then be adapted to propose a more structured and transparent fare system for Tbilisi, Georgia.

### Summary of EDA Insights and Hypotheses
From the exploratory data analysis, several key insights were derived:
- The fare amount, trip distance, trip duration, speed, and tip amount distributions were highly right-skewed, indicating the need for transformations.
- Fare amount had strong positive correlations with trip distance and trip duration.
- Categorical variables such as time of day, day type, season, and holidays significantly impacted fare amounts.
- Proper handling of outliers was crucial to prevent skewing model predictions.

#### Hypotheses
1. **Trip Distance and Duration:** Longer trips (both in terms of distance and duration) will result in higher fares.
2. **Temporal Factors:** Taxi fares vary significantly by time of day, day of the week, and season, with peak times and special events (e.g., holidays) resulting in higher fares.
3. **Speed:** Higher average speeds may correlate with higher fares due to longer distances covered in shorter times.
4. **Tips:** Higher fares tend to receive higher tips.

The modeling phase will test these hypotheses by evaluating the predictive power of these factors.


## Methodology

### Data Preparation
The dataset used for modeling has been cleaned and preprocessed based on the insights from the EDA. Key steps include:
- Log transformations to normalize skewed data.
- Handling outliers using Z-score filtering.
- Encoding categorical variables using one-hot encoding.
- Scaling numerical features.
- PCA analysis

### Model Selection
To build a robust fare prediction model, we will use both simple and complex machine learning models:
1. **Simple Models:**
   - Linear Regression
2. **Complex Models:**
   - Decision Trees
   - Random Forest
   - Gradient Boosting Machines (GBM)
   - XGBoost

### Model Evaluation Metrics
The performance of the models will be evaluated using the following metrics:
- Mean Absolute Error (MAE)
- Mean Squared Error (MSE)
- R-squared (R²)

These metrics will help in understanding the accuracy and reliability of the models.


In [42]:
# Import necessary libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split, RandomizedSearchCV




In [9]:
# Load the prepared dataset
df = pd.read_parquet('modeling_dataset_v.0.parquet')

In [10]:
df.head()

,fare_amount,trip_duration,trip_distance,total_amount,tip_amount,speed_mph,JFK_LGA_Pickup_Fee,General_Airport_Fee,log_fare_amount,log_trip_duration,...,pickup_time_of_day,pickup_season,passenger_count_category,pickup_day_type,PUzone,PUborough,DOzone,DOborough,PCA1,PCA2
19276873,19.1,20.366667,2.32,30.72,5.12,6.834697,0.0,0.0,3.000720,3.061832,...,afternoon,autumn,low,weekday,Midtown Center,Manhattan,East Village,Manhattan,-0.464965,0.734329
25505385,17.7,18.933333,1.90,25.20,3.50,6.021127,0.0,0.0,2.928524,2.992393,...,afternoon,autumn,low,weekday,Midtown East,Manhattan,East Chelsea,Manhattan,-0.787874,0.722135
23056552,8.6,8.950000,1.10,12.10,2.00,7.374302,0.0,0.0,2.261763,2.297573,...,afternoon,autumn,low,weekend,Manhattan Valley,Manhattan,East Harlem South,Manhattan,-1.516879,0.074052
22171336,10.0,8.966667,1.00,16.00,2.00,6.691450,0.0,0.0,2.397895,2.299246,...,afternoon,autumn,low,weekday,Midtown Center,Manhattan,Lincoln Square East,Manhattan,-1.523446,0.162344
25241061,7.9,5.783333,1.07,14.88,2.98,11.100865,0.0,0.0,2.186051,1.914469,...,afternoon,autumn,low,weekday,Upper West Side North,Manhattan,Manhattan Valley,Manhattan,-1.352724,-0.467095


In [11]:
len(df)

2805668

In [12]:
df.dtypes

fare_amount                  float64
trip_duration                float64
trip_distance                float64
total_amount                 float64
tip_amount                   float64
speed_mph                    float64
JFK_LGA_Pickup_Fee           float64
General_Airport_Fee          float64
log_fare_amount              float64
log_trip_duration            float64
log_trip_distance            float64
log_tip_amount               float64
is_holiday                     int64
payment_type                   int64
pickup_time_of_day          category
pickup_season               category
passenger_count_category    category
pickup_day_type             category
PUzone                      category
PUborough                   category
DOzone                      category
DOborough                   category
PCA1                         float64
PCA2                         float64
dtype: object

In [13]:


# Create a combined column for stratification
df['stratify_col'] = df['pickup_season'].astype(str) + '_' + df['pickup_time_of_day'].astype(str) + '_' + df['pickup_day_type'].astype(str) + '_' + df['PUborough'].astype(str) + '_' + df['DOborough'].astype(str)

# Check the distribution of the combined stratification column
stratify_counts = df['stratify_col'].value_counts()
print(stratify_counts)

# Filter out classes with very few samples
min_class_size = 5  # Define a minimum class size
filtered_classes = stratify_counts[stratify_counts >= min_class_size].index
df_filtered = df[df['stratify_col'].isin(filtered_classes)]

# Perform stratified sampling on the filtered dataset
df_sample, _ = train_test_split(df_filtered, test_size=0.9, stratify=df_filtered['stratify_col'], random_state=42)

# Drop the stratify column
df_sample = df_sample.drop(columns=['stratify_col'])


stratify_col
spring_evening_weekday_Manhattan_Manhattan      162096
spring_afternoon_weekday_Manhattan_Manhattan    152673
autumn_evening_weekday_Manhattan_Manhattan      149395
autumn_afternoon_weekday_Manhattan_Manhattan    140089
winter_afternoon_weekday_Manhattan_Manhattan    139463
                                                 ...  
autumn_night_weekend_Unknown_EWR                     1
autumn_afternoon_weekday_EWR_EWR                     1
autumn_evening_weekday_Unknown_EWR                   1
spring_night_weekday_Staten Island_Queens            1
winter_night_weekday_Unknown_EWR                     1
Name: count, Length: 1050, dtype: int64


In [23]:
print(len(df))
print(len(df_sample))

2805668
280513


For the modeling purposes we performed stratified sampling to ensure that all the categorical features remain intact and we have a model that can give us insights based on the categorical variables as well. 

In [ ]:
# Feature selection and encoding
features = df_sample[['PCA1', 'PCA2', 'is_holiday', 'pickup_time_of_day', 'pickup_season', 'passenger_count_category', 'pickup_day_type', 'PUzone', 'PUborough', 'DOzone', 'DOborough']]
target = df_sample['fare_amount']

categorical_features = ['is_holiday', 'pickup_time_of_day', 'pickup_season', 'passenger_count_category', 'pickup_day_type', 'PUzone', 'PUborough', 'DOzone', 'DOborough']
encoder = ColumnTransformer(transformers=[('cat', OneHotEncoder(drop='first'), categorical_features)], remainder='passthrough')

X = encoder.fit_transform(features)
y = target.values

# Reduce the number of features using SelectKBest
X_new = SelectKBest(f_regression, k=10).fit_transform(X, y)

# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_new, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize the data for KNN and SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_train_lr = linear_model.predict(X_train)
y_pred_val_lr = linear_model.predict(X_val)
print("Linear Regression Performance")
print("Training MAE:", mean_absolute_error(y_train, y_pred_train_lr))
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_lr))
print("Validation R²:", r2_score(y_val, y_pred_val_lr))

# Decision Tree with Grid Search for parameter tuning
param_grid_dt = {'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]}
dt_grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid_dt, cv=3, n_jobs=-1)
dt_grid_search.fit(X_train, y_train)
best_dt_model = dt_grid_search.best_estimator_
y_pred_val_dt = best_dt_model.predict(X_val)
print("Decision Tree Performance")
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_dt))
print("Validation R²:", r2_score(y_val, y_pred_val_dt))

# Random Forest with Grid Search for parameter tuning
param_grid_rf = {'n_estimators': [10, 50, 100], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]}
rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid_rf, cv=3, n_jobs=-1)
rf_grid_search.fit(X_train, y_train)
best_rf_model = rf_grid_search.best_estimator_
y_pred_val_rf = best_rf_model.predict(X_val)
print("Random Forest Performance")
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_rf))
print("Validation R²:", r2_score(y_val, y_pred_val_rf))

# Gradient Boosting with early stopping
gb_model = GradientBoostingRegressor(n_estimators=100, validation_fraction=0.1, n_iter_no_change=10)
gb_model.fit(X_train, y_train)
y_pred_val_gb = gb_model.predict(X_val)
print("Gradient Boosting Performance")
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_gb))
print("Validation R²:", r2_score(y_val, y_pred_val_gb))

# XGBoost with Grid Search for parameter tuning
param_grid_xgb = {'n_estimators': [50, 100], 'max_depth': [3, 6, 10], 'learning_rate': [0.01, 0.1]}
xgb_grid_search = GridSearchCV(xgb.XGBRegressor(random_state=42), param_grid_xgb, cv=3, n_jobs=-1)
xgb_grid_search.fit(X_train, y_train)
best_xgb_model = xgb_grid_search.best_estimator_
y_pred_val_xgb = best_xgb_model.predict(X_val)
print("XGBoost Performance")
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_xgb))
print("Validation R²:", r2_score(y_val, y_pred_val_xgb))

# K-Nearest Neighbors
knn_model = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)
knn_model.fit(X_train_scaled, y_train)
y_pred_val_knn = knn_model.predict(X_val_scaled)
print("KNN Performance")
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_knn))
print("Validation R²:", r2_score(y_val, y_pred_val_knn))

# Support Vector Machine
svm_model = SVR(kernel='rbf')
svm_model.fit(X_train_scaled, y_train)
y_pred_val_svm = svm_model.predict(X_val_scaled)
print("SVM Performance")
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_svm))
print("Validation R²:", r2_score(y_val, y_pred_val_svm))

# Summarize Model Performance
models = ['Linear Regression', 'Decision Tree', 'Random Forest', 'Gradient Boosting', 'XGBoost', 'KNN', 'SVM']
val_mae = [
    mean_absolute_error(y_val, y_pred_val_lr),
    mean_absolute_error(y_val, y_pred_val_dt),
    mean_absolute_error(y_val, y_pred_val_rf),
    mean_absolute_error(y_val, y_pred_val_gb),
    mean_absolute_error(y_val, y_pred_val_xgb),
    mean_absolute_error(y_val, y_pred_val_knn),
    mean_absolute_error(y_val, y_pred_val_svm)
]
val_r2 = [
    r2_score(y_val, y_pred_val_lr),
    r2_score(y_val, y_pred_val_dt),
    r2_score(y_val, y_pred_val_rf),
    r2_score(y_val, y_pred_val_gb),
    r2_score(y_val, y_pred_val_xgb),
    r2_score(y_val, y_pred_val_knn),
    r2_score(y_val, y_pred_val_svm)
]

performance_df = pd.DataFrame({
    'Model': models,
    'Validation MAE': val_mae,
    'Validation R²': val_r2
})

print(performance_df)

# Preparation For Modeling

In [29]:
# Feature selection and encoding
features = df_sample[['PCA1', 'PCA2', 'is_holiday', 'pickup_time_of_day', 'pickup_season', 'passenger_count_category', 'pickup_day_type', 'PUzone', 'PUborough', 'DOzone', 'DOborough']]
target = df_sample['fare_amount']

In [30]:
categorical_features = ['is_holiday', 'pickup_time_of_day', 'pickup_season', 'passenger_count_category', 'pickup_day_type', 'PUzone', 'PUborough', 'DOzone', 'DOborough']
encoder = ColumnTransformer(transformers=[('cat', OneHotEncoder(drop='first'), categorical_features)], remainder='passthrough')

In [32]:
X = encoder.fit_transform(features)
y = target.values

# Reduce the number of features using SelectKBest
X_new = SelectKBest(f_regression, k=10).fit_transform(X, y)

# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_new, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [35]:
# Standardize the data for KNN and SVM
scaler = StandardScaler(with_mean= False)
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


## Baseline Model

In [36]:
# Baseline Model: Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_train = linear_model.predict(X_train)
y_pred_val = linear_model.predict(X_val)

print("Linear Regression Performance")
print("Training MAE:", mean_absolute_error(y_train, y_pred_train))
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val))
print("Validation R²:", r2_score(y_val, y_pred_val))


Linear Regression Performance
Training MAE: 2.697683248247949
Validation MAE: 2.6931883885075365
Validation R²: 0.944835876489503



### Linear Regression Model

**Performance Metrics:**
- **Training MAE:** 2.70
- **Validation MAE:** 2.69
- **Validation R²:** 0.9448

**Interpretation:**
1. **Mean Absolute Error (MAE):**
   - The average absolute error in fare prediction is approximately $2.70 on the training data and $2.69 on the validation data. The closeness of these values indicates that the model is not overfitting and generalizes well to unseen data.
2. **R-squared (R²):**
   - The R² value of 0.9448 indicates that approximately 94.48% of the variance in taxi fares can be explained by the model. This high R² value suggests that the linear regression model fits the data well.

**Conclusion:**
- The linear regression model performs well with a high R² value and low MAE, indicating strong predictive capability for the given features. However, it is a simple model and may not capture complex, non-linear relationships in the data.



## Complex Models

In [39]:

# Decision Tree with Grid Search for parameter tuning
param_grid_dt = {'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]}
dt_grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid_dt, cv=3, n_jobs=-1)
dt_grid_search.fit(X_train, y_train)
best_dt_model = dt_grid_search.best_estimator_
y_pred_val_dt = best_dt_model.predict(X_val)
print("Decision Tree Performance")
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_dt))
print("Validation R²:", r2_score(y_val, y_pred_val_dt))


Decision Tree Performance
Validation MAE: 2.3716119571271497
Validation R²: 0.9545403292306762


### Decision Tree Model

**Performance Metrics:**
- **Validation MAE:** 2.37
- **Validation R²:** 0.9545

**Interpretation:**
1. **Mean Absolute Error (MAE):**
   - The average absolute error in fare prediction is approximately $2.37 on the validation data. This is slightly lower than the MAE of the linear regression model, indicating better predictive accuracy.
2. **R-squared (R²):**
   - The R² value of 0.9545 indicates that approximately 95.45% of the variance in taxi fares can be explained by the decision tree model. This is higher than the R² value for the linear regression model, suggesting that the decision tree captures more variability in the data.

**Conclusion:**
- The decision tree model performs better than the linear regression model in terms of both MAE and R². It captures more complex relationships between the features and the target variable. However, decision trees can be prone to overfitting, so it is essential to validate the model on unseen data and potentially prune the tree or limit its depth to prevent overfitting.


In [44]:
# Gradient Boosting with early stopping
gb_model = GradientBoostingRegressor(n_estimators=100, validation_fraction=0.1, n_iter_no_change=10)
gb_model.fit(X_train, y_train)
y_pred_val_gb = gb_model.predict(X_val)
print("Gradient Boosting Performance")
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_gb))
print("Validation R²:", r2_score(y_val, y_pred_val_gb))


Gradient Boosting Performance
Validation MAE: 2.41214002650482
Validation R²: 0.9550262508155019


In [45]:
# XGBoost with Grid Search for parameter tuning
param_grid_xgb = {'n_estimators': [50, 100], 'max_depth': [3, 6, 10], 'learning_rate': [0.01, 0.1]}
xgb_grid_search = GridSearchCV(xgb.XGBRegressor(random_state=42), param_grid_xgb, cv=3, n_jobs=-1)
xgb_grid_search.fit(X_train, y_train)
best_xgb_model = xgb_grid_search.best_estimator_
y_pred_val_xgb = best_xgb_model.predict(X_val)
print("XGBoost Performance")
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_xgb))
print("Validation R²:", r2_score(y_val, y_pred_val_xgb))

XGBoost Performance
Validation MAE: 2.3678238791443933
Validation R²: 0.9528451666259213


In [ ]:
# Random Forest with Randomized Search for parameter tuning using a smaller sample and fewer folds
param_dist_rf = {'n_estimators': [10, 50], 'max_depth': [10, 20], 'min_samples_split': [2, 5]}
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, test_size=0.9, random_state=42)
rf_random_search = RandomizedSearchCV(RandomForestRegressor(random_state=42), param_distributions=param_dist_rf, n_iter=10, cv=2, n_jobs=-1, random_state=42)
rf_random_search.fit(X_train_sample, y_train_sample)
best_rf_model = rf_random_search.best_estimator_
final_rf_model = RandomForestRegressor(**best_rf_model.get_params())
final_rf_model.fit(X_train, y_train)
y_pred_val_rf = final_rf_model.predict(X_val)
print("Random Forest Performance")
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_rf))
print("Validation R²:", r2_score(y_val, y_pred_val_rf))


In [47]:
# K-Nearest Neighbors
knn_model = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)
knn_model.fit(X_train_scaled, y_train)
y_pred_val_knn = knn_model.predict(X_val_scaled)
print("KNN Performance")
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_knn))
print("Validation R²:", r2_score(y_val, y_pred_val_knn))


KeyboardInterrupt: 

In [ ]:
# Support Vector Machine
svm_model = SVR(kernel='rbf')
svm_model.fit(X_train_scaled, y_train)
y_pred_val_svm = svm_model.predict(X_val_scaled)
print("SVM Performance")
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_svm))
print("Validation R²:", r2_score(y_val, y_pred_val_svm))


In [ ]:
# Neural Network (MLP)
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Predict on training and validation data
y_pred_train_nn = model.predict(X_train)
y_pred_val_nn = model.predict(X_val)

print("Neural Network Performance")
print("Training MAE:", mean_absolute_error(y_train, y_pred_train_nn))
print("Validation MAE:", mean_absolute_error(y_val, y_pred_val_nn))
print("Validation R²:", r2_score(y_val, y_pred_val_nn))


In [ ]:
models = ['Linear Regression', 'Decision Tree', 'Random Forest', 'Gradient Boosting', 'XGBoost', 'KNN', 'SVM', 'Neural Network']
train_mae = [
    mean_absolute_error(y_train, linear_model.predict(X_train)),
    mean_absolute_error(y_train, dt_model.predict(X_train)),
    mean_absolute_error(y_train, rf_model.predict(X_train)),
    mean_absolute_error(y_train, gb_model.predict(X_train)),
    mean_absolute_error(y_train, xgb_model.predict(X_train)),
    mean_absolute_error(y_train, knn_model.predict(X_train_scaled)),
    mean_absolute_error(y_train, svm_model.predict(X_train_scaled)),
    mean_absolute_error(y_train, model.predict(X_train))
]
val_mae = [
    mean_absolute_error(y_val, linear_model.predict(X_val)),
    mean_absolute_error(y_val, dt_model.predict(X_val)),
    mean_absolute_error(y_val, rf_model.predict(X_val)),
    mean_absolute_error(y_val, gb_model.predict(X_val)),
    mean_absolute_error(y_val, xgb_model.predict(X_val)),
    mean_absolute_error(y_val, knn_model.predict(X_val_scaled)),
    mean_absolute_error(y_val, svm_model.predict(X_val_scaled)),
    mean_absolute_error(y_val, model.predict(X_val))
]
val_r2 = [
    r2_score(y_val, linear_model.predict(X_val)),
    r2_score(y_val, dt_model.predict(X_val)),
    r2_score(y_val, rf_model.predict(X_val)),
    r2_score(y_val, gb_model.predict(X_val)),
    r2_score(y_val, xgb_model.predict(X_val)),
    r2_score(y_val, knn_model.predict(X_val_scaled)),
    r2_score(y_val, svm_model.predict(X_val_scaled)),
    r2_score(y_val, model.predict(X_val))
]

performance_df = pd.DataFrame({
    'Model': models,
    'Training MAE': train_mae,
    'Validation MAE': val_mae,
    'Validation R²': val_r2
})

print(performance_df)
